# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [3]:
import requests
import pyowm
import json

api_key = '4adb974f265efc471682a465e8ca91b7'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [5]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 291.85,
  'feels_like': 292.1,
  'temp_min': 290.88,
  'temp_max': 292.21,
  'pressure': 995,
  'humidity': 89,
  'sea_level': 995,
  'grnd_level': 991},
 'visibility': 10000,
 'wind': {'speed': 7.2, 'deg': 220},
 'clouds': {'all': 75},
 'dt': 1756890252,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1756876593,
  'sunset': 1756925015},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [ ]:
from pyowm.owm import OWM
owm = OWM('4adb974f265efc471682a465e8ca91b7')
weather_mgr = owm.weather_manager()

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [7]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 291.85°C, Humidity: 89%, Wind Speed: 7.2 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [15]:
from pyowm.owm import OWM
owm = OWM('4adb974f265efc471682a465e8ca91b7')
mgr = owm.weather_manager()
weather = mgr.weather_at_place('London,GB').weather
temp_dict_kelvin = weather.temperature()   # a dict in Kelvin units (default when no temperature units provided)
temp_dict_kelvin['temp_min']
temp_dict_kelvin['temp_max']
temp_dict_fahrenheit = weather.temperature('fahrenheit')  # a dict in Fahrenheit units
temp_dict_celsius = weather.temperature('celsius')  # guess?

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 291.85°C, Humidity: 89%, Wind Speed: 7.2 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [12]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [11]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-09-03 12:00:00, Temp: 290.44°C
Time: 2025-09-03 15:00:00, Temp: 294.19°C
Time: 2025-09-03 18:00:00, Temp: 298.25°C
Time: 2025-09-03 21:00:00, Temp: 296.54°C
Time: 2025-09-04 00:00:00, Temp: 293.65°C
Time: 2025-09-04 03:00:00, Temp: 294.28°C
Time: 2025-09-04 06:00:00, Temp: 292.95°C
Time: 2025-09-04 09:00:00, Temp: 292.18°C
Time: 2025-09-04 12:00:00, Temp: 293.01°C
Time: 2025-09-04 15:00:00, Temp: 296.65°C
Time: 2025-09-04 18:00:00, Temp: 298.15°C
Time: 2025-09-04 21:00:00, Temp: 296.54°C
Time: 2025-09-05 00:00:00, Temp: 295.86°C
Time: 2025-09-05 03:00:00, Temp: 295.51°C
Time: 2025-09-05 06:00:00, Temp: 293.1°C
Time: 2025-09-05 09:00:00, Temp: 293.38°C
Time: 2025-09-05 12:00:00, Temp: 293.68°C
Time: 2025-09-05 15:00:00, Temp: 297.37°C
Time: 2025-09-05 18:00:00, Temp: 300.39°C
Time: 2025-09-05 21:00:00, Temp: 301.06°C
Time: 2025-09-06 00:00:00, Temp: 300.05°C
Time: 2025-09-06 03:00:00, Temp: 297.8°C
Time: 2025-09-06 06:00:00, Temp: 297.15°C
Time: 2025-09-06 09:00:00, Temp: 296

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [18]:
from pyowm.owm import OWM
owm = OWM('3abf032d774491da5a90ded130c74b87')
mgr = owm.weather_manager()
daily_forecast = mgr.forecast_at_place('Berlin,DE', 'daily').forecast
three_h_forecast = mgr.forecast_at_place('Berlin,DE', '3h').forecast

UnauthorizedError: Invalid API Key provided

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [ ]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [ ]:
#your code here